# Car Tracking project by Emil Wåreus


In [6]:

import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
from scipy.ndimage.measurements import label
import matplotlib.image as mpimg
import time
from skimage.feature import hog 

## Extract Features


In [7]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                            vis=False, feature_vec=True):
        # Call with two outputs if vis==True
        if vis == True:
            features, hog_image = hog(img, orientations=orient, 
                                      pixels_per_cell=(pix_per_cell, pix_per_cell),
                                      cells_per_block=(cell_per_block, cell_per_block), 
                                      transform_sqrt=False, 
                                      visualise=vis, feature_vector=feature_vec)
            return features, hog_image
        # Otherwise call with one output
        else:      
            features = hog(img, orientations=orient, 
                           pixels_per_cell=(pix_per_cell, pix_per_cell),
                           cells_per_block=(cell_per_block, cell_per_block), 
                           transform_sqrt=False, 
                           visualise=vis, feature_vector=feature_vec)
            return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                            
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=(0, 256))
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=(0, 256))
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=(0, 256))
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


In [8]:
def features_car(image, orient, pix_per_cell, cell_per_block, spatial_feat = True, hist_feat = True, hog_feat = True,
             s_size=(32, 32), nbins = 32, hog_channel = 'ALL'):
    file_features = []
    if spatial_feat == True:
        spatial_features = bin_spatial(image, size=s_size)
        file_features.append(spatial_features)
        
    if hist_feat == True:  
        hist_features = color_hist(image, nbins=nbins)
        file_features.append(hist_features)
        
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(image.shape[2]):
                
                hog_features.append(get_hog_features(image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            feature_image = cv2.cvtColor(image, cv2.COLOR_LUV2RGB)
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            hog_features = get_hog_features(image[:,:], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        file_features.append(hog_features)
    return file_features

# Define a function to extract features from a list of images
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel='ALL'):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    n_imgs = len(imgs)
    i = 0
    for file in imgs:
        i = i + 1
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        
        features_file = features_car(image = feature_image, 
                                 orient = orient, 
                                 pix_per_cell = pix_per_cell, 
                                 cell_per_block = cell_per_block, 
                                 spatial_feat = True, 
                                 hist_feat = True, 
                                 hog_feat = True,
                                 s_size=(32, 32), 
                                 nbins = 32, 
                                 hog_channel = hog_channel)
        
            
        
        features.append(np.concatenate(features_file))
        
        fliped_image=cv2.flip(feature_image,1)
        features_file = features_car(image = fliped_image, 
                                 orient = orient, 
                                 pix_per_cell = pix_per_cell, 
                                 cell_per_block = cell_per_block, 
                                 spatial_feat = True, 
                                 hist_feat = True, 
                                 hog_feat = True,
                                 s_size=(32, 32), 
                                 nbins = 32, 
                                 hog_channel = hog_channel)  
        features.append(np.concatenate(features_file))
        if (i % int(n_imgs/10)) == 0:
            print(i, " of ", n_imgs, ".. ", int(100*(float(i)/n_imgs)), "%")
            
    # Return list of feature vectors
    return features

## Lets train a classifier!! 

In [9]:
#Load the data
import glob
cars = []
not_cars = []
for img in glob.glob('*data/non-vehicles/*/*.png'):
    not_cars.append(img)
    
    
for img in glob.glob("data/vehicles/*/*.png"):
    cars.append(img)


print("Number of cars : ", len(cars))
print("Number of none-cars : ", len(not_cars))

Number of cars :  8792
Number of none-cars :  8968


In [10]:
cspace='RGB'
orient=9
pix_per_cell=8
cell_per_block=2
hog_channel='ALL'

car_features = extract_features(imgs = cars, 
                                cspace=cspace, 
                                orient=orient, 
                                pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel)



not_cars_features = extract_features(imgs = not_cars, 
                                cspace=cspace, 
                                orient=orient, 
                                pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel)
    
print("Number of cars : ", len(car_features))
print("Number of none-cars : ", len(not_cars_features))


C:\ProgramData\Anaconda3\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


879  of  8792 .. in %:  9
1758  of  8792 .. in %:  19
2637  of  8792 .. in %:  29
3516  of  8792 .. in %:  39
4395  of  8792 .. in %:  49
5274  of  8792 .. in %:  59
6153  of  8792 .. in %:  69
7032  of  8792 .. in %:  79
7911  of  8792 .. in %:  89
8790  of  8792 .. in %:  99
896  of  8968 .. in %:  9
1792  of  8968 .. in %:  19
2688  of  8968 .. in %:  29
3584  of  8968 .. in %:  39
4480  of  8968 .. in %:  49
5376  of  8968 .. in %:  59
6272  of  8968 .. in %:  69
7168  of  8968 .. in %:  79
8064  of  8968 .. in %:  89
8960  of  8968 .. in %:  99
Number of cars :  17584
Number of none-cars :  17936


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

print(len(car_features) + len(not_cars_features))
use_data = 5000

# Building a classifier:
X = np.vstack((car_features[:use_data], not_cars_features[:use_data])).astype(np.float64)
print("X Done")
Y = np.hstack((np.ones(len(car_features[:use_data])), np.zeros(len(not_cars_features[:use_data]))))
print("Y Done")
X_scaler = StandardScaler().fit(X) # Fit a per-column scaler
scaled_X = X_scaler.transform(X) # Apply the scaler to X

X_train, X_test, Y_train, Y_test = train_test_split(scaled_X, Y, test_size = 0.2, random_state=22)


35520
X Done
Y Done


In [ ]:
#Train support vector Machine
SVC = LinearSVC(loss = 'hinge')
SVC.fit(X_train, Y_train)
print("Accuracy of SVC : ", round(SVC.score(X_test, Y_test), 5))
